In [3]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [16]:
df_costs = pd.read_csv('https://raw.githubusercontent.com/ZorroHZR/MMAI-5000/main/Omis%20A3/costs.csv')


# Extract data from DataFrame
fixed_cost_establishment_costs = df_costs['Fixed'].tolist()
variable_cost_transportation_costs = df_costs['Variable'].tolist()

# Create a new model
model = gp.Model("HealthLink_Distribution_Network")

# Create decision variables
site_units = {}
site_selection = {}

for site_id in range(27):
    site_units[site_id] = model.addVar(lb=0, vtype=GRB.INTEGER, name=f"units_site_{site_id}")
    site_selection[site_id] = model.addVar(vtype=GRB.BINARY, name=f"selected_site_{site_id}")

# Set objective function
def calculate_total_cost():
    establishment_costs = sum(fixed_cost_establishment_costs[site_id] * site_selection[site_id] for site_id in range(27))
    transportation_costs = sum(variable_cost_transportation_costs[site_id] * site_units[site_id] for site_id in range(27))
    return establishment_costs + transportation_costs

model.setObjective(calculate_total_cost(), GRB.MINIMIZE)

# Add constraints
model.addConstr(sum(site_units.values()) == 2900000, name="total_inventory_constraint")

for site_id in range(27):
    model.addConstr(175000 * site_selection[site_id] <= site_units[site_id], name=f"min_units_site_{site_id}")
    model.addConstr(site_units[site_id] <= 375000 * site_selection[site_id], name=f"max_units_site_{site_id}")

model.addConstr(sum(site_selection[site_id] for site_id in range(5, 16)) >= 4, name="min_sites_6_16_constraint")
model.addConstr(sum(site_selection[site_id] for site_id in range(1, 27, 2)) <= 6, name="max_even_sites_constraint")

for i in range(2):
    for j in range(4, 7):
        model.addConstr(site_selection[i] + site_selection[j] <= 1, name=f"site_{i+1}_excludes_{j+1}_constraint")

for i in range(18, 22):
    for j in [23, 25, 26]:
        model.addConstr(site_selection[i] + site_selection[j] <= 1, name=f"site_{i+1}_excludes_{j+1}_constraint")

for i in range(5):
    model.addConstr(site_selection[i] <= sum(site_selection[j] for j in range(20, 27, 2)), name=f"site_{i+1}_implies_odd_21_27_constraint")

model.addConstr(sum(site_selection[i] for i in range(14)) == sum(site_selection[i] for i in range(14, 27)), name="equal_selected_sites_constraint")
model.addConstr(sum(site_units[i] for i in range(9)) == sum(site_units[i] for i in range(18, 27)), name="equal_allocated_units_constraint")

# Solve the model
model.optimize()

# Print the results
print(f"Optimal cost: ${model.objVal:.2f}")

# Calculate establishment costs and transportation costs
total_establishment_costs = sum(fixed_cost_establishment_costs[site_id] * site_selection[site_id].x for site_id in range(27))
total_transportation_costs = sum(variable_cost_transportation_costs[site_id] * site_units[site_id].x for site_id in range(27))

print(f"Establishment costs: ${total_establishment_costs:.2f}")
print(f"Transportation costs: ${total_transportation_costs:.2f}")

print("Selected sites:")
for site_id in range(27):
    if site_selection[site_id].x > 0.5:
        print(f"Site {site_id + 1}: {site_units[site_id].x} units")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 82 rows, 54 columns and 265 nonzeros
Model fingerprint: 0x9dfff1d4
Variable types: 0 continuous, 54 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+05]
  Objective range  [2e-01, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+06]
Found heuristic solution: objective 1.676125e+07
Presolve time: 0.00s
Presolved: 82 rows, 54 columns, 265 nonzeros
Variable types: 0 continuous, 54 integer (27 binary)
Found heuristic solution: objective 1.676125e+07

Root relaxation: objective 1.201633e+07, 40 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node T